# Hotel Review Classification

### Team 1: Renetta Nelson, Jacqueline Urenda

### June 26, 2023

## Importing Libraries

In [82]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import time
import random
from collections import OrderedDict
import string
import re


from random import randint
from time import sleep



# Functions

In [83]:


def extract(h_page, hotel_names, hotel_ratings, hotel_reviews, hotel_prices):

    h_content = BeautifulSoup(h_page.text, 'html.parser')

    for hotel_name in h_content.find_all('div', {'class': 'listing_title'}):
        hotel_names.append(hotel_name.text.strip())


    for hotel_rating in h_content.find_all('a', {'class': 'ui_bubble_rating'}):
        hotel_ratings.append(hotel_rating['alt'])

    for hotel_review in h_content.find_all('a', {'class': 'review_count'}):
        hotel_reviews.append(hotel_review.text.strip())

    for hotel_price in h_content.find_all('div', {'class': 'price-wrap'}):
        print("In pricing")
        hotel_prices.append(hotel_price.text.replace('₹','').strip())
    

    return hotel_names, hotel_ratings, hotel_reviews, hotel_prices


def extract_test(hotel_names, hotel_ratings, hotel_reviews, hotel_prices):
    print("Lengths of Dataset Columns")
    print("Hotel Names: ", len(hotel_names))
    print("Hotel Ratings: ", len(hotel_ratings))
    print("Hotel Reviews: ", len(hotel_reviews))
    print("Hotel Prices: ", len(hotel_prices))




def processing(hotel_names, hotel_ratings, hotel_reviews):
    hotel_names2 = []
    hotel_names3 = []
    hotel_reviews2 = []
    hotel_ratings2 = []
    name_index = []
    num = 0

    # removing duplicates
    #print(hotel_names)
    for i in hotel_names:
        #print(i)
        if i not in hotel_names2:
            #print(i)
            x = i.split(" ")
            #print(x)
            if x[0] == "Sponsored":
                name_index.append(num)
                num = num + 1
                continue
            else:
                hotel_names2.append(" ".join(x))
                num = num + 1
        else:
            continue
    #print(hotel_names2)  



    # Converting reviews and ratings to numerical values

    q = 0
    #print(name_index)
    #print(len(hotel_reviews))
    for x in name_index:
        #print(x)
        if q % 2 == 0:
            #print(q)
            q = q + 1
            #print("x", x)
            hotel_reviews.remove(hotel_reviews[x])
            #print(q)
        else:
            q = q + 1
            continue


    for i in hotel_reviews:
        x = i.split(" ")
        for y in x:
            c = re.sub(r'[^\w\s]', '', y)
            if c.isdigit():
                hotel_reviews2.append(c)
            else:
                continue

    hotel_reviews2 = [eval(i) for i in hotel_reviews2]


    # Hotel Ratings 

    p = 0
    for x in name_index:
        #print(x)
        if p % 2 == 0:
            #print(p)
            p = p + 1
            hotel_ratings.remove(hotel_ratings[x])
            #print(p)
        else:
            p = p + 1
            continue
        

    for i in hotel_ratings:
        x = i.split(" ")
        hotel_ratings2.append(x[0])

    hotel_ratings2 = [eval(i) for i in hotel_ratings2]


    # removing numbers from hotel names

    for i in hotel_names2:
        x = i.split(" ")
        del x[0]
        hotel_names3.append(" ".join(x))

    #print(name_index)
    #print(hotel_ratings2)

    return hotel_names3, hotel_ratings2, hotel_reviews2



def processing_test(hotel_names3, hotel_ratings2, hotel_reviews2):
    print("Lengths of Dataset Columns")
    print("Hotel Names: ", len(hotel_names3))
    print("Hotel Ratings: ", len(hotel_ratings2))
    print("Hotel Reviews: ", len(hotel_reviews2))



def extract_review(url):
    
    hotel_rev = []

    review_page= (url)

    user_agent = ( {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \ AppleWebKit/537.36 (KHTML, like Gecko) \ Chrome/90.0.4430.212 Safari/537.36',
                    'Accept-Language': 'en-US, en'})


    rp1= requests.get(review_page, headers = user_agent)

    h_content = BeautifulSoup(rp1.content, 'html.parser')

    for rp_1 in h_content.findAll('span', {'class':'QewHA H4 _a'}):
        hotel_rev.append(rp_1.span.text.strip())
    
    return hotel_rev



## Retrieving Data from Website

In [84]:


hotel_page1 = ("https://www.tripadvisor.in/Hotels-g60750-San_Diego_California-Hotels.html")
hotel_page2 = ("https://www.tripadvisor.in/Hotels-g60750-oa30-San_Diego_California-Hotels.html")
hotel_page3 = ("https://www.tripadvisor.in/Hotels-g60750-oa60-San_Diego_California-Hotels.html")
hotel_page4 = ("https://www.tripadvisor.in/Hotels-g60750-oa90-San_Diego_California-Hotels.html")
hotel_page5 = ("https://www.tripadvisor.in/Hotels-g60750-oa120-San_Diego_California-Hotels.html")
hotel_page6 = ("https://www.tripadvisor.in/Hotels-g60750-oa150-San_Diego_California-Hotels.html")
hotel_page7 = ("https://www.tripadvisor.in/Hotels-g60750-oa180-San_Diego_California-Hotels.html")
hotel_page8 = ("https://www.tripadvisor.in/Hotels-g60750-oa210-San_Diego_California-Hotels.html")
hotel_page9 = ("https://www.tripadvisor.in/Hotels-g60750-oa240-San_Diego_California-Hotels.html")
hotel_page10 = ("https://www.tripadvisor.in/Hotels-g60750-oa270-San_Diego_California-Hotels.html")

user_agent = ( {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \ AppleWebKit/537.36 (KHTML, like Gecko) \ Chrome/90.0.4430.212 Safari/537.36',
                'Accept-Language': 'en-US, en'})


hp1= requests.get(hotel_page1, headers = user_agent)
hp2= requests.get(hotel_page2, headers = user_agent)
hp3= requests.get(hotel_page3, headers = user_agent)
hp4= requests.get(hotel_page4, headers = user_agent)
hp5= requests.get(hotel_page5, headers = user_agent)
hp6= requests.get(hotel_page6, headers = user_agent)
hp7= requests.get(hotel_page7, headers = user_agent)
hp8= requests.get(hotel_page8, headers = user_agent)
hp9= requests.get(hotel_page9, headers = user_agent)
hp10= requests.get(hotel_page10, headers = user_agent)




## Extracting the Data

In [85]:

hotel_names = []
hotel_ratings = []
hotel_reviews = []
hotel_prices = []


hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp1, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp2, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp3, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp4, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp5, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp6, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp7, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp8, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp9, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)
#hotel_names, hotel_ratings, hotel_reviews, hotel_prices = extract(hp10, hotel_names, hotel_ratings, hotel_reviews, hotel_prices)


#extract_test(hotel_names, hotel_ratings, hotel_reviews, hotel_prices)


In [86]:
extract_test(hotel_names, hotel_ratings, hotel_reviews, hotel_prices)

Lengths of Dataset Columns
Hotel Names:  60
Hotel Ratings:  30
Hotel Reviews:  30
Hotel Prices:  0


## Data Preparation

In [87]:

hotel_names3, hotel_ratings2, hotel_reviews2 = processing(hotel_names, hotel_ratings, hotel_reviews)

#processing_test(hotel_names3, hotel_ratings2, hotel_reviews2)

## Loading Data 

In [88]:

hotels_dict = {}

# Load data into dictionary

hotels_dict = {'names':hotel_names3, 'num_reviews':hotel_reviews2, 'ratings':hotel_ratings2}   #, 'costs':hotel_prices[:37]}


hotels_df = pd.DataFrame.from_dict(hotels_dict)
hotels_df.head(10)


,names,num_reviews,ratings
0,Bahia Resort Hotel,6349,4.5
1,Manchester Grand Hyatt San Diego,12617,4.5
2,Embassy Suites by Hilton San Diego Bay Downtown,3429,4.5
3,San Diego Mission Bay Resort,1354,4.0
4,Catamaran Resort Hotel and Spa,6744,4.5
5,Paradise Point Resort & Spa,3180,4.0
6,San Diego Marriott La Jolla,1502,4.5
7,Best Western Plus Island Palms Hotel & Marina,4851,4.5
8,Old Town Inn,2412,4.5
9,Urban Boutique Hotel,1332,4.5


## Extracting Hotel Reviews

In [89]:

hotel_rev = BeautifulSoup(hp1.content, 'html.parser')

links = []
for review in hotel_rev.findAll('a',{'class': 'review_count'}):
    a = review['href']
    a = 'https://www.tripadvisor.in'+ a

    a= a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
  
    links.append(a)

    #links


In [90]:
reviews = []

for link in links:
    reviews.append(extract_review(link))
    
#reviews

## Loading Reviews into Data Frame

In [91]:

hotels_df["reviews"] = reviews

hotels_df.head(10)


,names,num_reviews,ratings,reviews
0,Bahia Resort Hotel,6349,4.5,"[Such a beautiful resort, very relaxing, and s..."
1,Manchester Grand Hyatt San Diego,12617,4.5,[Loved my trip. Was treated with immense care!...
2,Embassy Suites by Hilton San Diego Bay Downtown,3429,4.5,[Great breakfast. Surprised by a great happy h...
3,San Diego Mission Bay Resort,1354,4.0,"[Family friendly resort, very close to Sea Wor..."
4,Catamaran Resort Hotel and Spa,6744,4.5,[We had our 2023 Company Retreat here at the C...
5,Paradise Point Resort & Spa,3180,4.0,"[To share the highlights, the grounds were imp..."
6,San Diego Marriott La Jolla,1502,4.5,[Thanks you Stephanie and Gerry for a wonderfu...
7,Best Western Plus Island Palms Hotel & Marina,4851,4.5,[We had a quick 2.5 day trip to San Diego and ...
8,Old Town Inn,2412,4.5,[Booked this for our San Diego trip. The sink ...
9,Urban Boutique Hotel,1332,4.5,[Great location in the middle of everything bu...
